<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_Ensemble_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retriever

sparse / Dense

앙상블 : 2가지 방식을 써서... 더 정확한 답을 얻을 수 있도록 한다.
롱컨텍스트 : AI가 똑똑하군...

In [98]:
# # colab 환경설정
# import os
# from google.colab import userdata

# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
# os.environ['HF_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

# from google.colab import drive as gd
# drive.mount('/content/drive')

In [99]:
# pip install --quiet icecream > /dev/null

In [100]:
# import pprint
# from icecream import ic

# def pp(object):
#   ppr = pprint.PrettyPrinter(
#       # indent=40,
#       width=80
#       )
#   return ppr.pprint(object)

# Ensenble

In [101]:
# pip install -q langchain langchain-openai pypdf sentence-transformers chromadb  faiss-cpu -U rank_bm25 > /dev/null

In [102]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
ko_embed = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sbert-nli",
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
    )

In [104]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

SOURCE_folder = "/content/drive/MyDrive/data/brand"
SOURCE = "/content/source/1-1그로스 해킹, 마케팅과 어떻게 다른가요_ - PUBLY.pdf"

loader = DirectoryLoader(
    SOURCE_folder,
    glob='**/*.pdf',
    show_progress=True,
    loader_cls=PyPDFLoader,
    )

pages = loader.load_and_split()

# 이건 여러번의 로더 호출을 통해, 각 문서가 각각의 로더객체에 담겼을 경우,
# 이 모두를 합해서 하나의 doc 객체로 만들어서 downstream에서 이용하기 위해 합쳐주는 기능.
# 따라서... 우리는 디렉토리 로더로 어차피 하나의 객체에 모든 문서를 '페이지별'로 담았다.
# 이 pypdf는 내부적으로 리컬시브~를 쓴다고 한다.
# docs = []
# for page in pages:
#     docs.append(page.page_content)

ic(len(pages))

100%|██████████| 3/3 [00:03<00:00,  1.00s/it]
ic| len(pages): 47, len(pages_02): 47


(47, 47)

In [105]:
# 이걸 문서별로 덩어리로 만들고 싶다. 즉 불러온 문서가 3개라면, 객체속의 요소도 3개만 존재하도록.
# keys = []
# for page in pages:
#     source = page.metadata['source']
#     if source not in keys:
#         keys.append(page.metadata['source'])
# keys

# 그러나 지금은 일단 포기

lens = []
for page in pages:
    lens.append(len(page.page_content))

print(sorted(lens, reverse = True), end=' ')
print('\n','*'*100, '\n')
print(min(lens), max(lens))

docs = pages

[1498, 1409, 1283, 1240, 1236, 1225, 1217, 1202, 1187, 1175, 1161, 1134, 1120, 1105, 1085, 1079, 977, 945, 909, 904, 857, 812, 801, 795, 794, 781, 776, 769, 757, 752, 734, 729, 727, 708, 701, 676, 675, 635, 626, 592, 585, 472, 446, 446, 407, 320, 281] 
 **************************************************************************************************** 

281 1498


In [106]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 적정한 길이로 잘라주고
rc_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=30,
    length_function=len,
)

# chunks = rc_splitter.create_documents(docs) # 옵션을 줄 수 있다. []을 받는다.
chunks = rc_splitter.split_documents(docs) # 옵션을 줄 수 있다. []을 받는다.

len(chunks)

lens = []
for chunk in chunks:
    lens.append(len(chunk.page_content))

print(sorted(lens, reverse = True), end=' ')
print('\n','*'*100, '\n')
print(min(lens), max(lens))

[350, 349, 349, 348, 347, 347, 346, 346, 346, 345, 344, 343, 343, 343, 343, 343, 342, 341, 340, 340, 340, 339, 339, 339, 338, 338, 338, 337, 337, 336, 335, 335, 335, 334, 334, 334, 334, 334, 334, 334, 334, 334, 334, 333, 333, 332, 332, 331, 331, 330, 330, 330, 329, 329, 328, 328, 327, 327, 327, 326, 325, 324, 324, 323, 322, 322, 320, 319, 319, 319, 319, 319, 318, 318, 318, 317, 317, 317, 315, 315, 314, 314, 314, 314, 313, 313, 312, 312, 311, 311, 310, 309, 309, 308, 307, 306, 305, 305, 304, 302, 300, 300, 300, 295, 294, 290, 281, 279, 258, 255, 250, 245, 243, 238, 233, 224, 221, 214, 202, 201, 197, 194, 191, 185, 147, 146, 145, 145, 143, 142, 142, 139, 129, 128, 125, 114, 109, 106, 99, 87, 84, 83, 78, 70, 69] 
 **************************************************************************************************** 

69 350


In [106]:
bm25_ret = BM25Retriever.from_documents(chunks)
